# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import numpy
import graphlab

A newer version of GraphLab Create (v1.10) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('../kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

In [7]:
print sales

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 |  221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 |  538000.0 |   3.0    |    2.25   |
| 5631500400 | 2015-02-25 00:00:00+00:00 |  180000.0 |   2.0    |    1.0    |
| 2487200875 | 2014-12-09 00:00:00+00:00 |  604000.0 |   4.0    |    3.0    |
| 1954400510 | 2015-02-18 00:00:00+00:00 |  510000.0 |   3.0    |    2.0    |
| 7237550310 | 2014-05-12 00:00:00+00:00 | 1225000.0 |   4.0    |    4.5    |
| 1321400060 | 2014-06-27 00:00:00+00:00 |  257500.0 |   3.0    |    2.25   |
| 2008000270 | 2015-01-15 00:00:00+00:00 |  291850.0 |   3.0    |    1.5    |
| 2414600126 | 2015-04-15 00:00:00+00:00 |  229500.0 |   3.0    |    1.0    |
| 3793500160 | 2015-03-12 00:00:00+00:00 |  323000.0 |   3.0    

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [4]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [5]:
def get_numpy_data(feature_data, feature_names, target_feature):
    """
    feature_data: SFrame mapping feature names to data values
    feature_names: list of feature names
    target_feature: target feature name
    """
    # Add new data element 'constant' with value 1; this is for w0 (intercept)
    feature_data['constant'] = 1 
    
    # Select 'constant' and specified feature columns into feature_sframe
    selected_feature_data = feature_data.select_columns(['constant'] + feature_names)
    
    # Select target_feature values into output_sarray
    output_values = feature_data[target_feature]
    
    # Return numpy matrix and array corresponding to feature sframe and output array
    return (selected_feature_data.to_numpy(), output_values.to_numpy())

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    """
    feature_matrix: numpy matrix of feature data
    weights: numpy array of feature weights
    """
    return feature_matrix.dot(weights)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [13]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [14]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [15]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [7]:
def normalize_features(feature_matrix):
    """
    feature_matrix: numpy matrix of feature data
    
    return (normalized_matrix, norms)
    """
    norms = np.linalg.norm(feature_matrix, axis=0)
    return feature_matrix/norms, norms

To test the function, run the following:

In [8]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [10]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [11]:
normalized_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [15]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [16]:
predictions = predict_output(normalized_feature_matrix, weights)

In [17]:
print predictions

[ 0.02675867  0.04339256  0.01990703 ...,  0.02289873  0.03178473
  0.02289873]


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [18]:
def calculate_ro(i, feature_matrix, output, predictions, weights):
    f_vector = feature_matrix[:,i]
    return np.sum(f_vector*((output-predictions + weights[i]* f_vector)))

In [31]:
ro_val_0 = calculate_ro(0, normalized_feature_matrix, output, predictions, weights)
ro_val_1 = calculate_ro(1, normalized_feature_matrix, output, predictions, weights)
ro_val_2 = calculate_ro(2, normalized_feature_matrix, output, predictions, weights)

print ro_val_0
print ro_val_1
print ro_val_2

79400300.0349
87939470.773
80966698.676


In [21]:
num_cols = normalized_feature_matrix.shape[1]
ro_vals = []
for i in range(0, num_cols):
    ro_vals.append(calculate_ro(i, normalized_feature_matrix, output, predictions, weights))

In [22]:
for i, ro_val in enumerate(ro_vals):
    print("{0}: {1}".format(simple_features[i-1] if i>0 else 'intercept', ro_val))

intercept: 79400300.0349
sqft_living: 87939470.773
bedrooms: 80966698.676


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [26]:
def min_lambda_for_ro(ro):
    return ro*-2
def max_lambda_for_ro(ro):
    return ro*2

In [33]:
min_lambda_0 = min_lambda_for_ro(ro_val_0)
max_lambda_0 = max_lambda_for_ro(ro_val_0)
min_lambda_1 = min_lambda_for_ro(ro_val_1)
max_lambda_1 = max_lambda_for_ro(ro_val_1)
min_lambda_2 = min_lambda_for_ro(ro_val_2)
max_lambda_2 = max_lambda_for_ro(ro_val_2)

print("w_0 zero for {0:.2E} < lambda < {1:.2E}".format(min_lambda_0, max_lambda_0))
print("w_1 zero for {0:.2E} < lambda < {1:.2E}".format(min_lambda_1, max_lambda_1))
print("w_2 zero for {0:.2E} < lambda < {1:.2E}".format(min_lambda_2, max_lambda_2))

w_0 zero for -1.59E+08 < lambda < 1.59E+08
w_1 zero for -1.76E+08 < lambda < 1.76E+08
w_2 zero for -1.62E+08 < lambda < 1.62E+08


In [17]:
lambda_threshold_1 = 2*ro_vals[1]
lambda_threshold_2 = 2*ro_vals[2]

In [18]:
print lambda_threshold_1
print lambda_threshold_2

175878941.546
161933397.352


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [181]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    predictions = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = calculate_ro(i, feature_matrix, output, predictions, weights)
  
    lambda_threshold = l1_penalty/2.
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -1*lambda_threshold:
        new_weight_i = ro_i + lambda_threshold
    elif ro_i > lambda_threshold:
        new_weight_i = ro_i - lambda_threshold
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [182]:
# should print 0.425558846691
import math

m = np.array([[3./math.sqrt(13), 1./math.sqrt(10)], [2./math.sqrt(13), 3./math.sqrt(10)]])
o = np.array([1., 1.])
w = np.array([1., 4.])
l1_penalty = 0.1
print lasso_coordinate_descent_step(1, m, o, w, l1_penalty)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [183]:
import copy
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_iterations = 200
    converged = False
    iterations = 0
    weights = copy.copy(initial_weights)
    while not converged and iterations < max_iterations:
        iterations += 1
        max_step_size = None
        for i in range(len(weights)):
            orig_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            step_size = abs(orig_weight - weights[i])
            if max_step_size is None or step_size > max_step_size:
                max_step_size = step_size
        if iterations%10 == 0:        
            print("[{0}] max_step_size={1:.2E}".format(iterations, max_step_size))
        converged = max_step_size < tolerance
        if converged:
            print("Converged with max_step_size = {0:.2E}".format(max_step_size))

    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [184]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [185]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [186]:
optimized_weights = lasso_cyclical_coordinate_descent(
    normalized_simple_feature_matrix, 
    output,
    np.zeros(3),
    l1_penalty, 
    tolerance)

[10] max_step_size=2.27E+06
[20] max_step_size=3.82E+05
[30] max_step_size=6.43E+04
[40] max_step_size=1.08E+04
[50] max_step_size=1.82E+03
[60] max_step_size=3.07E+02
[70] max_step_size=5.17E+01
[80] max_step_size=8.71E+00
[90] max_step_size=1.47E+00
Converged with max_step_size = 8.60E-01


In [187]:
for i, w in enumerate(optimized_weights):
    print("{0}: {1}".format(simple_features[i-1] if i>0 else 'intercept', w))

intercept: 21624998.3664
sqft_living: 63157246.7855
bedrooms: 0.0


In [151]:
print optimized_weights
print simple_features

[ 21624998.36636292  63157246.78545421         0.        ]
['sqft_living', 'bedrooms']


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

In [188]:
optimized_predictions = predict_output(normalized_simple_feature_matrix, optimized_weights)
residuals = optimized_predictions - output
RSS = (residuals.dot(residuals)).sum()
print('%.2E' % RSS)

1.63E+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [130]:
train_data, test_data = sales.random_split(.8, seed=0)

Let us consider the following set of features.

In [189]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [144]:
(all_feature_matrix, all_feature_output) = get_numpy_data(train_data, all_features, 'price')
(normalized_all_feature_matrix, all_feature_norms) = normalize_features(all_feature_matrix) 

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [197]:
num_features = len(all_features)
weights_1e7 = lasso_cyclical_coordinate_descent(
    normalized_all_feature_matrix, 
    all_feature_output,
    np.zeros(num_features+1),
    1e7, 
    1.)

[10] max_step_size=2.71E+06
[20] max_step_size=2.23E+05
[30] max_step_size=4.03E+04
[40] max_step_size=7.31E+03
[50] max_step_size=1.32E+03
[60] max_step_size=2.40E+02
[70] max_step_size=4.34E+01
[80] max_step_size=7.86E+00
[90] max_step_size=1.42E+00
Converged with max_step_size = 8.53E-01


In [198]:
for i, w in enumerate(weights_1e7):
    if w != 0:
        print("{0}: {1}".format(all_features[i-1] if i>0 else 'intercept', w))

intercept: 24429600.6093
sqft_living: 48389174.3523
waterfront: 3317511.16272
view: 7329961.9849


***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [199]:
weights_1e8 = lasso_cyclical_coordinate_descent(
    normalized_all_feature_matrix, 
    all_feature_output,
    np.zeros(num_features+1),
    1e8, 
    1.)

Converged with max_step_size = 0.00E+00


In [202]:
for i, w in enumerate(weights_1e8):
    if w != 0:
        print("{0}: {1}".format(all_features[i-1] if i>0 else 'intercept', w))

intercept: 71114625.7528


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [203]:
weights_1e4 = lasso_cyclical_coordinate_descent(
    normalized_all_feature_matrix, 
    all_feature_output,
    np.zeros(num_features+1),
    1e4, 
    1.)

[10] max_step_size=1.67E+06
[20] max_step_size=1.16E+06
[30] max_step_size=1.08E+06
[40] max_step_size=9.59E+05
[50] max_step_size=8.28E+05
[60] max_step_size=7.14E+05
[70] max_step_size=6.19E+05
[80] max_step_size=5.46E+05
[90] max_step_size=4.90E+05
[100] max_step_size=4.47E+05
[110] max_step_size=4.23E+05
[120] max_step_size=4.13E+05
[130] max_step_size=4.02E+05
[140] max_step_size=3.91E+05
[150] max_step_size=3.79E+05
[160] max_step_size=3.67E+05
[170] max_step_size=3.55E+05
[180] max_step_size=3.44E+05
[190] max_step_size=3.33E+05
[200] max_step_size=3.22E+05


In [204]:
for i, w in enumerate(weights_1e4):
    if w != 0:
        print("{0}: {1}".format(all_features[i-1] if i>0 else 'intercept', w))

intercept: 86227959.2313
bedrooms: -21111518.6704
bathrooms: 8753680.50069
sqft_living: 130999328.698
sqft_lot: -1986853.70587
floors: -6284165.96517
waterfront: 6590240.20596
view: 6494170.87813
condition: 11316702.1308
grade: 53322021.0346
sqft_above: -53729383.2344
sqft_basement: -15085545.1717
yr_built: -126114586.371
yr_renovated: 2931209.74949


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [205]:
def get_normalized_weights(l1_penalty):
    (feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
    (norm_feature_matrix, norms) = normalize_features(feature_matrix) 
    weights = lasso_cyclical_coordinate_descent(
        norm_feature_matrix, 
        output,
        np.zeros(len(all_features)+1),
        l1_penalty,
        1.)
    return weights/norms

In [206]:
normalized_weights1e7 = get_normalized_weights(1e7)

[10] max_step_size=2.71E+06
[20] max_step_size=2.23E+05
[30] max_step_size=4.03E+04
[40] max_step_size=7.31E+03
[50] max_step_size=1.32E+03
[60] max_step_size=2.40E+02
[70] max_step_size=4.34E+01
[80] max_step_size=7.86E+00
[90] max_step_size=1.42E+00
Converged with max_step_size = 8.53E-01


In [207]:
print normalized_weights1e7[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [208]:
normalized_weights1e8 = get_normalized_weights(1e8)

Converged with max_step_size = 0.00E+00


In [209]:
normalized_weights1e4 = get_normalized_weights(1e4)

[10] max_step_size=1.67E+06
[20] max_step_size=1.16E+06
[30] max_step_size=1.08E+06
[40] max_step_size=9.59E+05
[50] max_step_size=8.28E+05
[60] max_step_size=7.14E+05
[70] max_step_size=6.19E+05
[80] max_step_size=5.46E+05
[90] max_step_size=4.90E+05
[100] max_step_size=4.47E+05
[110] max_step_size=4.23E+05
[120] max_step_size=4.13E+05
[130] max_step_size=4.02E+05
[140] max_step_size=3.91E+05
[150] max_step_size=3.79E+05
[160] max_step_size=3.67E+05
[170] max_step_size=3.55E+05
[180] max_step_size=3.44E+05
[190] max_step_size=3.33E+05
[200] max_step_size=3.22E+05


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [215]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [216]:
def calculate_rss(normalized_weights):
    predictions = predict_output(test_feature_matrix, normalized_weights)
    residuals = predictions - test_output
    return (residuals.dot(residuals)).sum()

In [217]:
rss_1e7 = calculate_rss(normalized_weights1e7)
rss_1e8 = calculate_rss(normalized_weights1e8)
rss_1e4 = calculate_rss(normalized_weights1e4)

In [218]:
print("rss 1e7: {0} ({0:.2E})".format(rss_1e7))
print("rss 1e8: {0} ({0:.2E})".format(rss_1e8))
print("rss 1e4: {0} ({0:.2E})".format(rss_1e4))

rss 1e7: 2.75962079909e+14 (2.76E+14)
rss 1e8: 5.37166150034e+14 (5.37E+14)
rss 1e4: 2.14773677565e+14 (2.15E+14)


***QUIZ QUESTION***

Which model performed best on the test data?